# App store market analysis (Amit)


This project performs several analysis on Apple and Google's app stores to understand the market of free apps. It also sizes them to understand which kind of apps make the most sense to invest in.

## ==== Data ====

We work on data provided to us on:
1. Apple app store in 'AppleStore.csv'(Source: [Mobile App Store](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps))
2. Google play store in 'googleplaystore.csv' (Source: [Google Play Store Apps](https://www.kaggle.com/lava18/google-play-store-apps))

## ==== All import statements ====

In [1]:
from csv import reader
import copy

## ==== All functions ====

Function `explore_data` allows the user to explore any specific rows of dataset. (Optional) It also let's the user get the count of rows and columns. There is no return value.

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
        
    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset_slice[0]))

Function `open_dataset` allows users to open a dataset and returns the dataset as a **list**. (Optional) Idicate if there is a header. If indicated, the function will not return it. If you want the header indicate `False`. 

In [3]:
def open_dataset(csv_name, header=False):
    open_file = open(csv_name, encoding='utf8')
    read_file = reader(open_file)
    all_data = list(read_file)
    
    if header:
        all_data = all_data[1:]
        
    return all_data

Function `missing_data_rows` compares length of each row in data with the length of header and if the length is smaller it returns their index in a list. (Optional) You can ask the function to delete these rows from dataset. In this case we also return the new dataset.

In [4]:
def missing_data_rows(dataset, delete=False):
    header = dataset[0]
    rows = dataset[1:]
    index_list = []
    recombined_data = []
    
    for row in rows:
        if len(header) != len(row):
            index_list.append(rows.index(row))
            if delete:
                del rows[rows.index(row)]
    if delete:
        # Do not forget to recombine header with rows. Header will
        # also need to become a list of list, since rows is one
        recombined_data = [header] + rows
        return index_list, recombined_data
    
    return index_list

Function `duplicate_data` compares column_to_check for duplicates and returns a list of indexes with a dict object as details of it.

In [5]:
def check_duplicate_data(dataset, column_to_check):
    header = dataset[0]
    rows = dataset[1:]
    col_dict = {}
    index_list = []
    
    # Create a frequency table of items
    for row in rows:
        if row[column_to_check] not in col_dict:
            col_dict[row[column_to_check]] = 1
        else:
            col_dict[row[column_to_check]] += 1
            index_list.append(rows.index(row))

    # remove non duplicates from dictionary
    # we need to make a copy of col_dict because when we use pop
    # while looping it causes an error
    # we will pop from return_dict but loop from col
    return_dict = copy.deepcopy(col_dict)
    for x in col_dict:
        if col_dict[x] == 1:
            return_dict.pop(x)
            
    return index_list, return_dict  

Function `delete_duplicate_rows` will delete duplicates (If we examine the rows we identified as duplicates, the main difference happens on the fourth column of each row, which corresponds to the number of reviews. The different numbers show that the data was collected at different times. We can use this to build a criterion for keeping rows. We won't remove rows randomly, but rather we'll keep the rows that have the highest number of reviews because the higher the number of reviews, the more reliable the ratings.)

In [6]:
def delete_duplicate_rows(dataset, col_to_check, review_col):
    new_dataset=[]
    header = dataset[0]
    rows = dataset[1:]
    review_dict = {}
    index_list = []
    
    # Create a frequency table of items
    for row in rows:
        if row[col_to_check] not in review_dict:
            review_dict[row[col_to_check]] = float(row[review_col])
        elif row[col_to_check] in review_dict:
             if float(review_dict[row[col_to_check]]) < float(row[review_col]):
                    review_dict[row[col_to_check]] = float(row[review_col])
    
    # dump rows into new_dataset
    new_dataset = [header]
        
    for row in rows:        
        if row[col_to_check] not in review_dict:
            new_dataset.append(row)
        else:
            # Check if this is the heighest reviews
            if float(row[review_col]) == review_dict[row[col_to_check]]:
                # since there are duplicate rows of same reviews
                # we need to do this once only
                # we have to extract names in the list of list
                # and check if name is in it
                chk_list = []
                for item in new_dataset:
                    chk_list.append(item[0])
                    
                # short cut for writing above three lines in one line
                # chk_list = [item[0] for item in new_dataset] 
                
                if row[col_to_check] not in chk_list:
                    new_dataset.append(row)

    return new_dataset

Function `check_non_english` returns apps with charachters where `ord()` value is not between `0` and `127`. It expects these to be forigen language apps.

In [7]:
def check_non_english(dataset, name_column):
    header = dataset[0]
    rows = dataset[1:]
    
    non_eng_dict = {}
    
    for row in rows:
        for char in row[name_column]:
            if (0 < ord(char) > 127):
                if row[name_column] not in non_eng_dict:
                    non_eng_dict[row[name_column]] = 1
                if row[name_column] in non_eng_dict:
                    non_eng_dict[row[name_column]] += 1
    
    return_dict = copy.deepcopy(non_eng_dict)
    
    for name in non_eng_dict:
        # Since English apps use Emojis etc, we want to save data
        # we are deleting apps which have more than 3 chars with
        # ord values not between 0 to 127
        if non_eng_dict[name] < 4:
            return_dict.pop(name)
    
    return return_dict

Function `delete_non_english` deletes non-english apps from the dataset and returns a new dataset

In [8]:
def delete_non_english(dataset, name_column):
    header = dataset[0]
    rows = dataset[1:]
    
    non_eng_dict = {}
    english_duct = {}
    
    new_dataset = [header]
    
    for row in rows:
        # put in non-english
        for char in row[name_column]:
            if (0 < ord(char) > 127):
                if row[name_column] not in non_eng_dict:
                    non_eng_dict[row[name_column]] = 1
                if row[name_column] in non_eng_dict:
                    non_eng_dict[row[name_column]] += 1
        # put in english if frequency < 4
        # append if not found
        if row[name_column] not in non_eng_dict:
            new_dataset.append(row)
        elif row[name_column] in non_eng_dict:
            if non_eng_dict[row[name_column]] < 4:
                new_dataset.append(row)
            
    return new_dataset

Function `check_paid_apps` returns the list of paid apps, which will be deleted from the dataset.

In [9]:
def check_paid_apps(dataset, name_col, price_col):
    header = dataset[0]
    rows = dataset[1:]
    
    paid_apps_dict = {}
    
    for row in rows:
        if price_col == 4:
            # treatment of apple (their column doesn't have $ in string)
            if float(row[price_col]) != 0.0:
                if row[name_col] not in paid_apps_dict:
                    paid_apps_dict[row[name_col]] = float(row[price_col])
        elif price_col == 7:
            # treatment for Google (the price col has $ in the string)
            price = float(row[price_col].replace('$',''))
            if float(price) > 0.0:
                if row[name_col] not in paid_apps_dict:
                    paid_apps_dict[row[name_col]] = float(price)
            
    
    return paid_apps_dict

Function `delete_paid_apps` deletes paid apps from the dataset

In [10]:
def delete_paid_apps(dataset, name_col, price_col):
    header = dataset[0]
    rows = dataset[1:]
    
    new_dataset = []
    new_rows = []
    for row in rows:
        if price_col == 4:
            # treatment of apple (their column doesn't have $ in string)
            if float(row[price_col]) == 0.0 or row[price_col] is None:
                new_rows.append(row)
                
        elif price_col == 7:
            # treatment for Google (the price col has $ in the string)
            price = float(row[price_col].replace('$',''))
            if float(price) == 0.0 or price is None:
                new_rows.append(row)

    new_dataset = [header] + new_rows 
    return new_dataset

Function `gen_fq_table` generates a dictionary object with % of frequency of that column.

In [11]:
def gen_fq_table(dataset, column_index):
    header = dataset[0]
    rows = dataset[1:]
    fq_dict = {}
    
    for row in rows:
        if row[column_index] not in fq_dict:
            fq_dict[row[column_index]] = 1
        else:
            fq_dict[row[column_index]] += 1
    
    for item in fq_dict:
        fq_dict[item] = round(((fq_dict[item] / len(rows)) * 100),2) 
    
    sorted_dict = display_table(fq_dict)
    
    return sorted_dict

Function `display_table` sorts a dictionary and returns a tuple. It also flips the keys and values, i.e. Values take first columns and Keys take the second one.

In [12]:
def display_table(dict_to_sort):
    # Optional sort by frequency
    table_display = []
    for key in dict_to_sort:
        key_val_as_tuple = (dict_to_sort[key], key)
        table_display.append(key_val_as_tuple)
        
    sorted_dict = sorted(table_display, reverse=True)   
    #sorted_dict = sorted(fq_dict.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_dict

## ==== Execution code ====

### 01 -> Testing file open, and exploring the datasets.

In [13]:
# Apple
apple_file = 'AppleStore.csv'

apple_data = open_dataset(apple_file, False)
explore_data(apple_data, 0, 3, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows: 7198
Number of columns: 16


In [14]:
# Google
google_file = 'googleplaystore.csv'

google_data = open_dataset(google_file, False)
explore_data(google_data, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


Number of rows: 10842
Number of columns: 13


### 02 -> Data cleaning

1. **Rows with missing entries or errors**
2. Remove duplicates
3. Remove non-english apps
4. Isolate free apps

In [15]:
# fix Apple data
apple_error_index = missing_data_rows(apple_data, delete=False)
apple_error_index

[]

In [16]:
if len(apple_error_index) > 0:
    apple_error_index, apple_data = missing_data_rows(apple_data, delete=True)

In [17]:
# fix Google data
google_error_index = missing_data_rows(google_data, delete=False)
google_error_index

[10472]

In [18]:
if len(google_error_index) > 0:
    google_error_index, google_data = missing_data_rows(google_data, delete=True)

In [19]:
print('Apple Data:', len(apple_data))
print('Google Data:', len(google_data))

Apple Data: 7198
Google Data: 10841


### 02 -> Data cleaning (contd.)

1. Rows with missing entries or errors
2. **Remove duplicates**
3. Remove non-english apps
4. Isolate free apps

In [20]:
apple_duplicate_index, apple_duplicate_dict = check_duplicate_data (apple_data, 0)

In [21]:
print(apple_duplicate_dict)
print('\n\n')
print(len(apple_duplicate_index), 'duplicate(s) found')

{}



0 duplicate(s) found


In [22]:
google_duplicate_index, google_duplicate_dict = check_duplicate_data (google_data, 0)

In [23]:
print(google_duplicate_dict)
print('\n\n')
print(len(google_duplicate_index), 'duplicate(s) found')

{'Coloring book moana': 2, 'Mcqueen Coloring pages': 2, 'UNICORN - Color By Number & Pixel Art Coloring': 2, 'Textgram - write on photos': 2, 'Wattpad 📖 Free Books': 2, 'Amazon Kindle': 2, 'Dictionary - Merriam-Webster': 2, 'NOOK: Read eBooks & Magazines': 2, 'Oxford Dictionary of English : Free': 2, 'Spanish English Translator': 2, 'NOOK App for NOOK Devices': 2, 'Ebook Reader': 2, 'English Dictionary - Offline': 2, 'Docs To Go™ Free Office Suite': 2, 'Google My Business': 3, 'OfficeSuite : Free Office + PDF Editor': 2, 'Curriculum vitae App CV Builder Free Resume Maker': 2, 'Facebook Pages Manager': 2, 'Box': 3, 'Call Blocker': 2, 'ZOOM Cloud Meetings': 2, 'Facebook Ads Manager': 2, 'Quick PDF Scanner + OCR FREE': 3, 'SignEasy | Sign and Fill PDF and other Documents': 2, 'Genius Scan - PDF Scanner': 2, 'Tiny Scanner - PDF Scanner App': 2, 'Fast Scanner : Free PDF Scan': 2, 'Mobile Doc Scanner (MDScan) Lite': 2, 'TurboScan: scan documents and receipts in PDF': 2, 'Tiny Scanner Pro: PD

**Start removing duplicates**

Apple's duplicates are in `0` and reviews are in `5` column. This is the column we will use to determine which duplicate rows to delete.

In [24]:
apple_nodups_data = delete_duplicate_rows(apple_data, 0, 5)


In [25]:
print('Original Apple data length:', len(apple_data))
print('Identified duplicates were:', len(apple_duplicate_index))
print('New Apple data length:', len(apple_nodups_data))

Original Apple data length: 7198
Identified duplicates were: 0
New Apple data length: 7198


In [26]:
explore_data(apple_nodups_data, 0, 3, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows: 7198
Number of columns: 16


Google's duplicates are in `0` and reviews are in `3` column. This is the column we will use to determine which duplicate rows to delete.

In [27]:
google_nodups_data = delete_duplicate_rows(google_data, 0, 3)


In [28]:
print('Original Google data length:', len(google_data))
print('Identified duplicates were:', len(google_duplicate_index))
print('New Google data length:', len(google_nodups_data))

Original Google data length: 10841
Identified duplicates were: 1181
New Google data length: 9660


In [29]:
explore_data(google_nodups_data, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 9660
Number of columns: 13


### 02 -> Data cleaning (contd.)

1. Rows with missing entries or errors
2. Remove duplicates
3. **Remove non-english apps**
4. Isolate free apps

**Info:** The numbers corresponding to the characters we commonly use in an English text are all in the range 0 to 127, according to the ASCII (American Standard Code for Information Interchange) system. Based on this number range, we can build a function that detects whether a character belongs to the set of common English characters or not. If the number is equal to or less than 127, then the character belongs to the set of common English characters.

If an app name contains a character that is greater than 127, then it probably means that the app has a non-English name.

Apple app names are in `1` column

In [30]:
apple_non_english = check_non_english(apple_nodups_data, 1)

In [31]:
print(apple_non_english)
print('\n\n')
print(len(apple_non_english), 'Suspected non-english app(s) found')

{'爱奇艺PPS -《欢乐颂2》电视剧热播': 14, '聚力视频HD-人民的名义,跨界歌王全网热播': 18, '优酷视频': 5, '网易新闻 - 精选好内容，算出你的兴趣': 17, '淘宝 - 随时随地，想淘就淘': 12, '搜狐视频HD-欢乐颂2 全网首播': 12, '阴阳师-全区互通现世集结': 12, '百度贴吧-全球最大兴趣交友社区': 15, '百度网盘': 5, '爱奇艺HD -《欢乐颂2》电视剧热播': 14, '乐视视频HD-白鹿原,欢乐颂,奔跑吧全网热播': 18, '万年历-值得信赖的日历黄历查询工具': 17, '新浪新闻-阅读最新时事热门头条资讯视频': 19, '喜马拉雅FM（听书社区）电台有声小说相声英语': 21, '央视影音-海量央视内容高清直播': 15, '腾讯视频HD-楚乔传,明日之子6月全网首播': 17, '手机百度 - 百度一下你就得到': 13, '百度视频HD-高清电视剧、电影在线观看神器': 19, 'MOMO陌陌-开启视频社交,用直播分享生活': 16, 'QQ 浏览器-搜新闻、选小说漫画、看视频': 17, '同花顺-炒股、股票': 9, '聚力视频-蓝光电视剧电影在线热播': 16, '快看漫画': 5, '乐视视频-白鹿原,欢乐颂,奔跑吧全网热播': 18, '酷我音乐HD-无损在线播放': 11, '随手记（专业版）-好用的记账理财工具': 18, 'Dictionary ( قاموس عربي / انجليزي + ودجيت الترجمة)': 29, '滴滴出行': 5, '淘宝HD－Taobao for iPad': 4, '高德地图（精准专业的手机地图）': 16, '百度HD-极速安全浏览器': 10, '美丽说-潮流穿搭快人一步': 12, '百度地图-智能的手机导航，公交地铁出行必备': 21, 'Majiang Mahjong（单机+川麻+二人+武汉+国标）': 13, '土豆视频HD—高清影视综艺视频播放器': 17, '360手机卫士-超安全的来电防骚扰助手': 16, 'QQ浏览器HD-极速搜索浏览器': 11, '搜狗输入法-Sogou Keyboard': 6, '百度网盘 HD': 5, '大众点评-发现品质生活': 11, '讯飞输入法-智能语音输入和表情斗

Google app names are in `0` column

In [32]:
google_non_english = check_non_english(google_nodups_data, 0)

In [33]:
print(google_non_english)
print('\n\n')
print(len(google_non_english), 'Suspected non-english app(s) found')

{'Truyện Vui Tý Quậy': 4, 'Flame - درب عقلك يوميا': 13, 'At home - rental · real estate · room finding application such as apartment · apartment': 4, '乐屋网: Buying a house, selling a house, renting a house': 4, 'သိင်္ Astrology - Min Thein Kha BayDin': 6, 'РИА Новости': 11, '乗換NAVITIME\u3000Timetable & Route Search in Japan Tokyo': 4, '► MultiCraft ― Free Miner! 👍': 4, 'صور حرف H': 7, 'L.POINT - 엘포인트 [ 포인트, 멤버십, 적립, 사용, 모바일 카드, 쿠폰, 롯데]': 24, 'M3-X5-X6-M-İ3-İ8 RACİNG 2018': 4, 'RMEduS - 음성인식을 활용한 R 프로그래밍 실습 시스템': 19, 'AJ렌터카 법인 카셰어링': 10, 'Al Quran Free - القرآن (Islam)': 7, '中国語 AQリスニング': 9, 'Neuvaines et prières au Coeur Sacré de Jésus': 4, '日本AV历史': 5, 'Ay Yıldız Duvar Kağıtları': 6, 'বাংলা টিভি প্রো BD Bangla TV': 14, 'Cъновник BG': 8, 'CSCS BG (в български)': 11, '뽕티비 - 개인방송, 인터넷방송, BJ방송': 15, 'BL 女性向け恋愛ゲーム◆俺プリクロス': 17, 'SecondSecret ‐「恋を読む」BLノベルゲーム‐': 15, 'BL 女性向け恋愛ゲーム◆ごくメン': 15, 'あなカレ【BL】無料ゲーム': 12, '감성학원 BL 첫사랑': 8, 'BQ-መጽሐፍ ቅዱሳዊ ጥያቄዎች': 14, 'BS Calendar / Patro / पात्रो': 7, 'Vip

**Start removing non-english apps**

In [34]:
apple_english_nodupes = delete_non_english(apple_nodups_data, 1)

In [35]:
print('Original Apple data length:', len(apple_nodups_data))
print('Identified non-english were:', len(apple_non_english))
print('New Apple data length:', len(apple_english_nodupes))

Original Apple data length: 7198
Identified non-english were: 1042
New Apple data length: 6156


In [36]:
explore_data(apple_english_nodupes, 0, 3, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows: 6156
Number of columns: 16


In [37]:
google_english_nodupes = delete_non_english(google_nodups_data, 0)

In [38]:
print('Original Google data length:', len(google_nodups_data))
print('Identified non-english were:', len(google_non_english))
print('New Google data length:', len(google_english_nodupes))

Original Google data length: 9660
Identified non-english were: 62
New Google data length: 9598


In [39]:
explore_data(google_english_nodupes, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 9598
Number of columns: 13


### 02 -> Data cleaning (contd.)

1. Rows with missing entries or errors
2. Remove duplicates
3. Remove non-english apps
4. **Isolate free apps**

Apple has name in `1` price in `4` column

In [40]:
apple_paid_apps = check_paid_apps(apple_english_nodupes, 1, 4)

In [41]:
print(apple_paid_apps)
print('\n\n')
print(len(apple_paid_apps), 'Paid app(s) found')

{'Fruit Ninja Classic': 1.99, 'Clear Vision (17+)': 0.99, 'Minecraft: Pocket Edition': 6.99, 'Plants vs. Zombies': 0.99, 'Doodle Jump': 0.99, 'Draw Something': 2.99, 'Infinity Blade': 0.99, 'Geometry Dash': 1.99, 'Tiny Wings': 0.99, 'Traffic Rush': 0.99, 'Pocket God': 0.99, 'Plants vs. Zombies HD': 0.99, 'Infinity Blade II': 0.99, 'The Room': 0.99, 'Plague Inc.': 0.99, "Where's My Water?": 1.99, 'Call of Duty: Black Ops Zombies': 6.99, 'Sunday Lawn': 0.99, 'SCRABBLE Premium': 7.99, 'Flick Home Run !': 0.99, 'Terraria': 4.99, 'Scribblenauts Remix': 0.99, 'Zombieville USA 2': 0.99, 'Trivia Crack (No Ads)': 2.99, 'Solitaire by MobilityWare': 4.99, 'TuneIn Radio Pro - MLB Audiobooks Podcasts Music': 9.99, 'Call of Duty: Zombies': 4.99, 'Cut the Rope: Experiments': 0.99, 'Phase 10 Pro - Play Your Friends!': 1.99, 'Videoshop - Video Editor': 1.99, 'The Sims 3': 6.99, 'HB2 PLUS': 0.99, 'StickWars': 0.99, 'BATTLE BEARS: Zombies!': 0.99, 'Slayin': 0.99, 'Ski Safari': 0.99, 'Lane Splitter': 0.99

Google has name in `0` price in `7` column

In [42]:
google_paid_apps = check_paid_apps(google_english_nodupes, 0, 7)

In [43]:
print(google_paid_apps)
print('\n\n')
print(len(google_paid_apps), 'Paid app(s) found')

{'TurboScan: scan documents and receipts in PDF': 4.99, 'Tiny Scanner Pro: PDF Doc Scan': 4.99, 'Puffin Browser Pro': 3.99, 'Truth or Dare Pro': 1.49, 'Private Dating, Hide App- Blue for PrivacyHider': 2.99, 'Ad Blocker for SayHi': 3.99, 'AMBW Dating App: Asian Men Black Women Interracial': 7.99, 'Moco+ - Chat, Meet People': 3.99, 'Sago Mini Hat Maker': 3.99, 'Fuzzy Numbers: Pre-K Number Foundation': 5.99, 'Toca Life: Hospital': 3.99, 'My Talking Pet': 4.99, 'Meme Generator': 2.99, 'My CookBook Pro (Ad Free)': 3.49, 'Paprika Recipe Manager': 4.99, 'Pocket Yoga': 2.99, 'Meditation Studio': 3.99, 'Relax Melodies P: Sleep Sounds': 2.99, 'Clue': 1.99, 'The Room: Old Sins': 4.99, 'The Escapists': 4.99, 'Farming Simulator 18': 4.99, 'RollerCoaster Tycoon® Classic': 5.99, 'Call of Duty:Black Ops Zombies': 6.99, 'Star Wars ™: DIRTY': 9.99, 'Monument Valley 2': 4.99, 'Toca Mystery House': 3.99, 'My Little Princess : Stores': 2.99, 'Sago Mini Big City': 3.99, 'Pinkalicious Party': 2.99, 'Toca Bu

**Start removing paid apps**

In [44]:
apple_free_appdata = delete_paid_apps(apple_english_nodupes, 1, 4)

In [45]:
print('Original Apple data length:', len(apple_english_nodupes))
print('Identified paid were:', len(apple_paid_apps))
print('New Apple data length:', len(apple_free_appdata))

Original Apple data length: 6156
Identified paid were: 2952
New Apple data length: 3204


In [46]:
explore_data(apple_free_appdata, 0, 3, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows: 3204
Number of columns: 16


In [47]:
google_free_appdata = delete_paid_apps(google_english_nodupes, 0, 7)

In [48]:
print('Original Google data length:', len(google_english_nodupes))
print('Identified paid were:', len(google_paid_apps))
print('New Google data length:', len(google_free_appdata))

Original Google data length: 9598
Identified paid were: 749
New Google data length: 8849


In [49]:
explore_data(google_free_appdata, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 8849
Number of columns: 13


### 03 -> Analysis start

As we mentioned in the introduction, our goal is to determine the kinds of apps that are likely to attract more users because the number of people using our apps affect our revenue.

To minimize risks and overhead, our validation strategy for an app idea has three steps:

1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we develop it further.
3. If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

Because our end goal is to add the app on both Google Play and the App Store, we need to find app profiles that are successful in both markets. For instance, a profile that works well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by determining the most common genres for each market. For this, we'll need to build frequency tables for a few columns in our datasets.

a) **Analysis of most common genres in each market**


|Store|column_name|Index|
|------|------|------|
|Apple|prime_genre|`11`|
|Google|Category|`1`|

**Analyze the frequency table you generated for the prime_genre column of the App Store dataset.**

1. **What is the most common genre? What is the next most common?**
2. What other patterns do you see?
3. What is the general impression — are most of the apps designed for practical purposes (education, shopping, utilities, productivity, lifestyle) or more for entertainment (games, photo and video, social networking, sports, music)?
4. Can you recommend an app profile for the App Store market based on this frequency table alone? If there's a large number of apps for a particular genre, does that also imply that apps of that genre generally have a large number of users?

In [54]:
apple_analysis1 = gen_fq_table(apple_free_appdata, 11)
print(apple_analysis1[0])
print(apple_analysis1[1])
print(apple_analysis1[2])
print(apple_analysis1[3])
print(apple_analysis1[4])

(58.26, 'Games')
(7.84, 'Entertainment')
(5.0, 'Photo & Video')
(3.68, 'Education')
(3.31, 'Social Networking')


1. What is the most common genre? What is the next most common?
2. **What other patterns do you see?**
3. What is the general impression — are most of the apps designed for practical purposes (education, shopping, utilities, productivity, lifestyle) or more for entertainment (games, photo and video, social networking, sports, music)?
4. Can you recommend an app profile for the App Store market based on this frequency table alone? If there's a large number of apps for a particular genre, does that also imply that apps of that genre generally have a large number of users?

**Analyze the frequency table you generated for the Category and Genres column of the Google Play dataset.**

1. **What are the most common genres?**
2. What other patterns do you see?
3. Compare the patterns you see for the Google Play market with those you saw for the App Store market.
4. Can you recommend an app profile based on what you found so far? Do the frequency tables you generated reveal the most frequent app genres or what genres have the most users?